In [1]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [2]:
loader = PDFPlumberLoader(r"D:\Petramount\Courses\AI\LLM\Basic_Home_Remedies.pdf")
docs = loader.load()

# Check the number of pages
print("Number of pages in the PDF:",len(docs))

# Load the random page content
docs[1].page_content

Number of pages in the PDF: 3


'\uf0b7 Warm Lemon Water: Drink on an empty stomach to stimulate digestion.\n\uf0b7 Flaxseeds: Soak overnight and consume in the morning for fiber intake.\n\uf0b7 Aloe Vera Juice: Helps in smooth bowel movements.\n2.6 Skin Problems (Acne, Dry Skin, Sunburn)\n\uf0b7 Aloe Vera Gel: Apply fresh aloe vera gel to reduce acne and soothe burns.\n\uf0b7 Coconut Oil: Moisturizes dry skin and prevents infections.\n\uf0b7 Cucumber & Honey Face Pack: Hydrates skin and reduces irritation.\n2.7 Joint Pain & Inflammation\n\uf0b7 Turmeric & Black Pepper: Mix 1 tsp turmeric with a pinch of black pepper in warm\nmilk. Reduces inflammation.\n\uf0b7 Epsom Salt Bath: Soak in warm water with Epsom salt for muscle relaxation.\n\uf0b7 Ginger Tea: Acts as a natural pain reliever.\n2.8 Insomnia (Sleep Issues)\n\uf0b7 Chamomile Tea: Calms the mind and promotes sleep.\n\uf0b7 Warm Milk & Nutmeg: Drink before bed to improve sleep quality.\n\uf0b7 Lavender Essential Oil: Diffuse in the bedroom to relax the mind.\n3

In [3]:
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

c:\Users\deepa\.conda\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print("Number of chunks created: ", len(documents))

print(documents[0].page_content)

Number of chunks created:  7
Basic Home Remedies & Healthy Lifestyle
Guide
1. Introduction
Home remedies have been used for centuries to treat common ailments using natural ingredients. Combined with a healthy lifestyle, these remedies can support overall well-being and disease
prevention. 2. Home Remedies for Common Ailments
2.1 Cold & Flu
 Ginger Tea: Boil ginger slices in water, add honey and lemon. Helps relieve congestion
and sore throat.  Turmeric Milk: Mix 1 tsp turmeric in warm milk. Acts as an anti-inflammatory and
boosts immunity.  Steam Inhalation: Boil water, add eucalyptus oil, and inhale steam to clear nasal
congestion. 2.2 Cough
 Honey & Lemon: Mix 1 tbsp honey with warm water and lemon juice. Soothes the
throat and reduces cough.  Tulsi (Holy Basil) Tea: Boil tulsi leaves in water and drink. Helps with respiratory
issues. 2.3 Headache
 Peppermint Oil: Apply a few drops on the temples for relief.  Ginger Paste: Make a paste of ginger and water, apply on forehead. 

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # size of each chunk
    chunk_overlap=50,     # overlap between chunks
    separators=["\n\n", "\n", ".", " ", ""]  # fallback hierarchy
)

documents = text_splitter.split_documents(docs)

In [ ]:
print("Number of chunks created: ", len(documents))

print(documents[0].page_content)

In [5]:
# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store 
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:
# from langchain_community.llms import Ollama

# llm = Ollama(model="llama3.2:1b")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

llm = ChatOpenAI(openai_api_base = "http://localhost:1234/v1", openai_api_key = "lm_studio", model = "qwen3-0.6b")



In [ ]:
# 2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n

In [7]:
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

prompt = """
1. You are doctor
2. Use the following pieces of context to answer the question at the end.
3. Answer only by using the context and articulate it better, use bullet point and emoji if required
4. Keep the answer crisp and limited to 3,4 sentences.

Context: {context}

Question: {question}

Answer to the question:"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )

In [ ]:
print(qa("remedy for cough?")["result"])